In [1]:
import numpy as np
import sys

sys.path.append('../src/')

from nodecore import Node
from nodecore import Variable
import basic_math

In [2]:
basic_math.install_node_arithmetics()

In [3]:
x = Variable(5)

y = x ** 2 + 2 * x + 1

In [4]:
y.data

36

In [5]:
a = Variable(np.ones((3, 3), dtype=np.float32))

In [6]:
b = a + a

In [7]:
b.data

array([[ 2.,  2.,  2.],
       [ 2.,  2.,  2.],
       [ 2.,  2.,  2.]], dtype=float32)

## Linear

In [9]:
class Linear(object):
    def __init__(self, in_ch, out_ch):
        self.W = Variable(
            np.random.normal(0, 1 / out_ch, (out_ch, in_ch)).astype('f')
        )

    def forward(self, x):
        self.x = x
        self.y = Node(linear_forward, [self.x, self.W])
        return self.y

    def backward(self):
        self.W.grad += Node(linear_backward_W, [self.x, self.y.grad])
        self.x.grad += Node(linear_backward_x, [self.W, self.y.grad])
    
    def __call__(self, x):
        return self.forward(x)

def linear_forward(x, W):
    y = np.matmul(x, W.T)
    return y

def linear_backward_W(x, gy):
    gW = np.matmul(gy.T, x)
    return gW

def linear_backward_x(W, gy):
    gx = np.matmul(gy, W)
    return gx

In [10]:
l0 = Linear( 5, 10)
l1 = Linear(10, 10)
l2 = Linear(10,  3)

In [11]:
# 真の係数
W = np.random.random((3, 5)).astype(np.float32)

In [12]:
batchsize = 8
x = Variable(np.random.random((batchsize, 5)).astype(np.float32))
t = Variable(np.matmul(x.data, W.T))

In [14]:
h0 = l0(x)
h1 = l1(h0)
y  = l2(h1)

loss = (y - t) ** 2

In [15]:
loss.data

array([[ 0.7765637 ,  0.11049283,  0.73104632],
       [ 1.59783959,  0.48883584,  0.76882285],
       [ 1.49823725,  0.36093721,  0.75923532],
       [ 1.55899572,  0.35230374,  1.24337208],
       [ 1.62110364,  0.33213964,  1.00779235],
       [ 1.40715814,  0.38675347,  1.69120514],
       [ 2.54401493,  0.5453096 ,  2.0999763 ],
       [ 1.10922825,  0.193562  ,  0.66312635]], dtype=float32)